### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Bipolar_disorder/GSE92538'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # Based on the microarray (Affymetrix) data mentioned in the background information

# Variable Availability
trait_row = 2 if 'Bipolar Disorder' in [d.split(': ')[1] for d in sample_characteristics[2]] else None
age_row = 8 if any(d.startswith('age: ') for d in sample_characteristics[8]) else None
gender_row = 6 if any(d in ['gender: M', 'gender: F'] for d in sample_characteristics[6]) else None

# Data Type Conversion Functions

def convert_trait(value):
    val = value.split(': ')[1]
    return 1 if val == "Bipolar Disorder" else 0

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except ValueError:
        return None

def convert_gender(value):
    val = value.split(': ')[1]
    return 1 if val == "M" else 0

# Save cohort information
save_cohort_info('GSE92538', './preprocessed/Bipolar_disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Bipolar_disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Bipolar_disorder/trait_data/GSE92538.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
